In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten
from keras.layers import ELU
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
dataGen=ImageDataGenerator(rescale=1/255,validation_split=0.25)
train_dir='/kaggle/input/rice-image-dataset/Rice_Image_Dataset/'

In [4]:
train_itt=dataGen.flow_from_directory(directory=train_dir,batch_size=64,target_size=(100,100),subset='training')
test_itt=dataGen.flow_from_directory(directory=train_dir,batch_size=64,target_size=(100,100),subset='validation')

In [5]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation=ELU(),input_shape=(100,100,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64,(3,3),activation=ELU()))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128,(3,3),activation=ELU()))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())

model.add(Dense(512,activation=ELU()))
model.add(Dense(5,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [6]:
early_stop= EarlyStopping(monitor='val_loss',patience=3,mode='min',verbose=False)
model.fit(train_itt,validation_data=test_itt,epochs=10,callbacks=early_stop)

In [7]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

In [8]:
class_label={'Arborio': 0, 'Basmati': 1, 'Ipsala': 2, 'Jasmine': 3, 'Karacadag': 4}

In [9]:
def prediction(path):
    image=load_img(path,target_size=(100,100))
    plt.imshow(image)
    plt.show()
    image=img_to_array(image)
    image=image.reshape(1,100,100,3)
    result=model.predict(image)
    result=np.argmax(result)
    return print([key for key in class_label][result])

In [10]:
prediction('/kaggle/input/rice-image-dataset/Rice_Image_Dataset/Ipsala/Ipsala (4678).jpg')